In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#imports
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
pd.set_option('max_rows',1000)
sns.set(rc={'figure.figsize':(15,10)})
from wordcloud import WordCloud
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
df = pd.read_csv('data.csv')
df_year = pd.read_csv('data_by_year.csv')
df_by_genres = pd.read_csv('data_by_genres.csv')
df_w_genres = pd.read_csv('data_w_genres.csv')
df_artist = pd.read_csv('data_by_artist.csv')

In [ ]:
df.head()

In [ ]:
df_year.head()

In [ ]:
df_artist.head()

In [ ]:
df_by_genres.head()

In [ ]:
df_w_genres.head()

In [ ]:
df.info()

# EXPLANATORY DATA ANALYSIS

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df1 = df.copy()
df1['duration_ms'].head()

In [ ]:
df1['duration_ms'] = df1['duration_ms']/1000
df1.rename({'duration_ms':'duration_in_seconds'},axis=1,inplace=True)
df1.info()

**Data Visualizations**

In [ ]:
df.hist(figsize=(20,20))
plt.show()

In [ ]:
fig,ax = plt.subplots(5,3,figsize=(20,25))
sns.distplot(df1['valence'],ax=ax[0,0])
sns.distplot(df1['year'],ax=ax[0,1])
sns.distplot(df1['acousticness'],ax=ax[0,2])
sns.distplot(df1['danceability'],ax=ax[1,0])
sns.distplot(df1['duration_in_seconds'],ax=ax[1,1])
sns.distplot(df1['energy'],ax=ax[1,2])
#sns.distplot(df1['explicit'],ax=ax[2,0])
sns.distplot(df1['instrumentalness'],ax=ax[2,1])
sns.distplot(df1['key'],ax=ax[2,2])
sns.distplot(df1['liveness'],ax=ax[3,0])
sns.distplot(df1['loudness'],ax=ax[3,1])
sns.distplot(df1['mode'],ax=ax[3,2])
sns.distplot(df1['popularity'],ax=ax[4,0])
sns.distplot(df1['speechiness'],ax=ax[4,1])
sns.distplot(df1['tempo'],ax=ax[4,2])

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df1.corr(),annot=True)

In [ ]:
g_pn = df1.groupby("name")['popularity'].sum().sort_values(ascending=False)[:20]
axis = sns.barplot(g_pn.index, g_pn,palette='rocket')
axis.set_title('Top Tracks with Popularity')
axis.set_ylabel('Popularity')
axis.set_xlabel('Tracks')
plt.xticks(rotation = 90)

In [ ]:
g_ap = df1.groupby("artists")['popularity'].sum().sort_values(ascending=False)[:20]
axis = sns.barplot(g_ap.index, g_ap,palette='magma_r')
axis.set_title('Top Artists with Popularity')
axis.set_ylabel('Popularity')
axis.set_xlabel('Artists')
plt.xticks(rotation = 90)

In [ ]:
columns = ["acousticness","danceability","energy","speechiness","liveness","valence"]
plt.figure(figsize=(15,10))
for c in columns:
    x = df1.groupby('year')[c].mean()
    sns.lineplot(x.index,x,label=c)
plt.title('Audio characteristics over year')
plt.xlabel('Year')
plt.ylabel('Characteristics')
plt.show()

In [ ]:
g_an = df1.groupby('artists')['name'].count().sort_values(ascending=False)[:20]
axis = sns.barplot(g_an.index, g_an,palette='winter')
axis.set_title('Top artists with tracks')
axis.set_ylabel('Track count')
axis.set_xlabel('Artists')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
g_an = df1.groupby('artists')['danceability'].mean().sort_values(ascending=False)[:20]
axis = sns.barplot(g_an.index, g_an,palette='summer')
axis.set_title('Top artists with danceability')
axis.set_ylabel('danceability')
axis.set_xlabel('Artists')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
df1['year'].describe()

In [ ]:
bins = [1920,1960,2000,2020]
df1['year_bins'] = pd.cut(df1['year'],bins,labels=['20s-60s','60s-2000','2000-2020'])
df1['year_bins'].head(10)

In [ ]:
df1['year_bins'].value_counts()

In [ ]:
g_yp = df1.groupby('year_bins')['popularity'].mean().sort_values(ascending=False)[:20]
axis = sns.barplot(g_yp.index, g_yp,palette='autumn_r')
axis.set_title('popularity categories')
axis.set_xlabel('Categories')
axis.set_ylabel('popularity')
#plt.xticks(rotation = 90)
plt.show()

In [ ]:
cols = ["valence","popularity","acousticness","instrumentalness","speechiness","danceability" ]
sns.pairplot(df1[cols])
plt.show()

In [ ]:
df1.columns

In [ ]:
plt.figure()
d = df1[:10000]
sns.scatterplot('tempo','popularity',data=d)
plt.show()

In [ ]:
plt.figure()
sns.barplot('explicit','popularity',data=df1,palette='rocket_r')
plt.show()

In [ ]:
wrds1 = g_ap.index.str.split("(").str[0].value_counts().keys()
wc1 = WordCloud(scale=5,max_words=1000,colormap="rainbow",background_color="white").generate(" ".join(wrds1))
plt.imshow(wc1,interpolation="bilinear")
plt.axis("off")
plt.title("Artist Name for top 50 songs ",color='b')
plt.show()

In [ ]:
sns.barplot('key','popularity',data=df1)
plt.show()

# ML AND MODELLING

In [ ]:
enc = LabelEncoder()
df1['Artist_enc'] = enc.fit_transform(df1['artists'])
df1['name_enc'] = enc.fit_transform(df1['name'])
df1['year_bins'] = df1['year_bins'].map({'20s-60s':0,'60s-2000':1,'2000-2020':2})
#print(df1['Artist_enc'].value_counts(),df1['name_enc'].value_counts())
df1.head(10)

In [ ]:
cols = ['valence', 'Artist_enc', 'danceability',
       'duration_in_seconds', 'energy', 'explicit',
       'key', 'liveness', 'loudness', 'mode', 'name_enc', 'tempo', 'year_bins']
X = df1[cols]
y = df1['popularity']
X = X[:30000]
y = y[:30000]
print(X.shape,y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#models
rf = RandomForestRegressor(bootstrap=True,criterion='mse',random_state=42,max_depth=35,
                           n_estimators=2500,n_jobs=-1)
xgb = XGBRegressor( booster='gbtree', colsample_bylevel=1,colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=4, n_estimators=4500,
             n_jobs=4, nthread=None, objective='reg:linear',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1)
gbr = GradientBoostingRegressor( learning_rate=0.01,) 
dtr = DecisionTreeRegressor(criterion='mse',random_state=42,max_depth=35,
                           max_features='sqrt', min_samples_leaf=15, min_samples_split=10)
abr = AdaBoostRegressor(dtr,learning_rate=0.01)
cat =  CatBoostRegressor(learning_rate=0.1,eval_metric = 'RMSE',verbose=0)
lgb = LGBMRegressor()

In [ ]:
xgb.fit(X_train,y_train)
xgb_pred = xgb.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,xgb_pred)),mean_absolute_error(y_test,xgb_pred))

In [ ]:
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,rf_pred)),mean_absolute_error(y_test,rf_pred))

In [ ]:
lgb.fit(X_train,y_train)
lgb_pred = lgb.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,lgb_pred)),mean_absolute_error(y_test,lgb_pred))

In [ ]:
dtr.fit(X_train,y_train)
dtr_pred = dtr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,dtr_pred)),mean_absolute_error(y_test,dtr_pred))

In [ ]:
gbr.fit(X_train,y_train)
gbr_pred = gbr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,gbr_pred)),mean_absolute_error(y_test,gbr_pred))

In [ ]:
abr.fit(X_train,y_train)
abr_pred = abr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,abr_pred)),mean_absolute_error(y_test,abr_pred))

In [ ]:
cat.fit(X_train,y_train)
cat_pred = cat.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,cat_pred)),mean_absolute_error(y_test,cat_pred))

In [ ]:
output = pd.DataFrame({'Actual_Popularity':y_test,'Predicted_mean':np.round((cat_pred+xgb_pred+lgb_pred+rf_pred)/4,2)})
output.head(10)

In [ ]:
output.to_csv('output.csv',index=False)